In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [81]:
# Import datasets
Mov=pd.read_csv('movies.csv',sep=',')
Rat=pd.read_csv('ratings.csv')
Osc=pd.read_csv('the_oscar_award.csv')
## Now start cleaning
# Transforming movies dataset: extract the years
def ext_movname(tlist):
    lst=tlist.copy()
    year=tlist.copy()
    for i in range(len(tlist)):
        raw=tlist[i].rstrip()
        try:
            year[i]=int(raw[-5:-1])
            lst[i]=raw[:-7]
        except:
            year[i]=np.nan
            lst[i]=raw
    return [lst,year]
u=ext_movname(Mov['title'])
Mov['title']=u[0]
Mov['year']=u[1]
CMov=Mov[Mov.year>=2008]
CMov=CMov.reset_index(drop=True)
# Function for recorrect the film name
def recorr(string):
    if '(' in string:
        nstr=string.split(' (')[0]
    else:
        nstr=string
    if ',' not in nstr:
        return nstr
    else:
        lst=nstr.split(sep=', ')
        if lst[-1] not in {'The', 'A'}:
            return nstr
        else:
            nlst=lst[:-1]
            new=lst[-1]+' '
            for j in nlst:
                new=new+j
            return new
CMov['title']=CMov['title'].apply(recorr)
# Choose the ceremony year and file name columns
C_osc=Osc[['year_ceremony','film','year_film']].dropna().copy()
C_osc=C_osc.reset_index(drop=True)
C_osc['film']=C_osc['film'].apply(recorr)
for i in range(len(C_osc)):
    if C_osc['film'][i][-1]==';':
        u=C_osc['film'][i][-1]
        C_osc['film'][i]=u[:-1]
C_osc=C_osc[C_osc.year_ceremony>=2009]
C_osc=C_osc.drop_duplicates(subset=['film','year_film'], keep='first').reset_index(drop=True)

/Users/lihuayu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [82]:
D_osc={(C_osc['film'][i],C_osc['year_film'][i]):C_osc['year_ceremony'][i] for i in range(len(C_osc))}

lst=[]
for i in range(len(CMov['title'])):
    if (CMov['title'][i],CMov['year'][i]) in D_osc:
        lst.append(D_osc[(CMov['title'][i],CMov['year'][i])])
    else:
        lst.append(0)
CMov['Oscar']=lst    
CMov=CMov[CMov.Oscar!=0]

In [116]:
# Transfering timestamp: extract the year data
def extract_year(time_se):
    return time.localtime(time_se).tm_year
Ye=Rat['timestamp'].apply(extract_year)
Rat['timestamp']=Ye

In [118]:
# Merge CMov and Rat, and filter the available rating times
R_Mov=pd.merge(CMov,Rat,on=['movieId'])
Ava_rating=R_Mov.reset_index(drop=True)[['movieId','title','Oscar','timestamp','rating']]

In [124]:
prev=Ava_rating[Ava_rating.timestamp<=Ava_rating.Oscar][['movieId','rating']]
after=Ava_rating[Ava_rating.timestamp>Ava_rating.Oscar][['movieId','rating']]

In [152]:
# Now calculate for the parameters
gprev=prev.groupby('movieId',as_index=False)
Stat_prev=gprev.count()
meanp=gprev.mean()['rating']
Stat_prev['prev_mean']=meanp
varp=gprev.var().fillna(0)['rating']
Stat_prev['prev_var']=varp
Stat_prev=Stat_prev[['movieId','prev_mean','prev_var']]

In [154]:
# Now calculate for the parameters
gafter=after.groupby('movieId',as_index=False)
Stat_after=gafter.count()
meana=gafter.mean()['rating']
Stat_after['after_mean']=meana
vara=gafter.var().fillna(0)['rating']
Stat_after['after_var']=vara
Stat_after=Stat_after[['movieId','after_mean','after_var']]

In [157]:
# Now merge
Compare=pd.merge(Stat_prev,Stat_after,how='left',on=['movieId'])

In [159]:
Compare['mean_diff']=Compare['after_mean']-Compare['prev_mean']
Compare['var_diff']=Compare['after_var']-Compare['prev_var']

In [163]:
Compare.to_csv('Comp_change.csv')